# Live GPS Plotting from NMEA TCP Socket

This notebook connects to a local TCP socket streaming NMEA sentences from a GPS module, parses the data, and plots the live GPS track on an interactive map.

In [ ]:
import socket
from pynmeagps import NMEAReader
from ipyleaflet import Map, Polyline, Circle, LayerGroup
from IPython.display import display
import queue

# Configuration
TCP_HOST = 'localhost'
TCP_PORT = 12345  # Change to your NMEA TCP port

# Map setup
INIT_CENTER = (51.0, 10.0)  # Default center (Germany)
m = Map(center=INIT_CENTER, zoom=14)
track_layer = LayerGroup()
m.add(track_layer)
m.layout.height = '600px'
display(m)

# Store all points for the polyline
track_points = []

# Store latest marker (as a Circle)
latest_circle = None

## Helper: Parse latitude and longitude from NMEA messages
def parse_nmea_latlon(msg):
    '''
    Extract latitude and longitude from a pynmeagps NMEA message.
    Returns (lat, lon) or None if not available.
    '''
    if hasattr(msg, 'lat') and hasattr(msg, 'lon') and msg.lat is not None and msg.lon is not None:
        return (msg.lat, msg.lon)
    return None

import time

poly = None
first_point = True

sock = None
try:
    sock = socket.create_connection((TCP_HOST, TCP_PORT))
    sock_file = sock.makefile('r', encoding='ascii')
    while len(line := sock_file.readline()) > 0:
        msg = NMEAReader.parse(line)
        pos = parse_nmea_latlon(msg)
        if pos:
            track_points.append(pos)

            # Remove previous circle marker
            if latest_circle:
                track_layer.remove(latest_circle)

            # Add a new circle marker for the latest point
            latest_circle = Circle(location=pos, radius=6, color='blue', fill_color='blue', fill_opacity=0.8, weight=2)
            track_layer.add(latest_circle)

            # Remove previous polyline
            if poly:
                track_layer.remove(poly)

            # Draw polyline for the whole track
            poly = Polyline(locations=track_points, color='red', fill=False, weight=5)
            track_layer.add(poly)

            # Center map only on first datapoint
            if first_point:
                m.center = pos
                first_point = False

except KeyboardInterrupt:
    print('Stopped by user.')
finally:
    if sock:
        try:
            sock.shutdown(socket.SHUT_RDWR)
        except Exception:
            pass
        sock.close()
        print('Socket closed.')

In [ ]:
# Plot GPS track from a log file (NMEA sentences, one per line)
from pynmeagps import NMEAReader, NMEAMessage
from ipyleaflet import Map, Polyline, Circle, LayerGroup
from IPython.display import display

def parse_nmea_latlon_log(msg: NMEAMessage):
    if msg.msgID == 'GGA' and  msg.lat != '':
        return (msg.lat, msg.lon)
    return None

LOG_FILE = '/home/florian/data/recordings/WAS/2025-07-17_14-10-32_gps.log'  # Change to your log file path

track_points = []
with open(LOG_FILE, 'r', encoding='ascii') as f:
    for line in f:
        _, nmea_str = line.strip().split(';')
        msg = NMEAReader.parse(nmea_str)
        pos = parse_nmea_latlon_log(msg)
        if pos:
            track_points.append(pos)

if track_points:
    INIT_CENTER = track_points[0]
    m = Map(center=INIT_CENTER, zoom=14)
    track_layer = LayerGroup()
    m.add(track_layer)
    m.layout.height = '600px'
    poly = Polyline(locations=track_points, color='red', fill=False, weight=5)
    track_layer.add(poly)
    latest_circle = Circle(location=track_points[-1], radius=6, color='blue', fill_color='blue', fill_opacity=0.8, weight=2)
    track_layer.add(latest_circle)
    display(m)
else:
    print('No valid points found in log file.')